In [8]:
import pandas as pd
import numpy as np

# Load the datasets
df = pd.read_csv("USDA_dada.csv")
df2 = pd.read_csv("recommended_nutrients.csv")

# Default weights for nutrients
default_weights = {
    'Energy (kcal)': 5,
    'Protein (g)': 100,
    'Carbohydrate (g)': 40,
    'Sugars, total\n(g)': 0.01,
    'Fiber, total dietary (g)': 70,
    'Total Fat (g)': 1,
    'Fatty acids, total saturated (g)': 0.001,
    'Fatty acids, total monounsaturated (g)': 1,
    'Fatty acids, total polyunsaturated (g)': 1,
    'Cholesterol (g)': 0.001,
    'Vitamin A, RAE (g)': 60,
    'Thiamin (g)': 45,
    'Riboflavin (g)': 30,
    'Niacin (g)': 30,
    'Vitamin B-6 (g)': 40,
    'Folate, total (g)': 60,
    'Choline, total (g)': 20,
    'Vitamin B-12 (g)': 60,
    'Vitamin C (g)': 60,
    'Vitamin D (D2 + D3) (g)': 70,
    'Vitamin E (alpha-tocopherol) (g)': 50,
    'Vitamin K (phylloquinone) (g)': 50,
    'Calcium (g)': 50,
    'Phosphorus (g)': 45,
    'Magnesium (g)': 20,
    'Iron\n(g)': 50,
    'Zinc\n(g)': 50,
    'Copper (g)': 50,
    'Selenium (g)': 20,
    'Potassium (g)': 40,
    'Sodium (g)': 1,
    'Caffeine (g)': 0.1,
    'Theobromine (g)': 5,
}

# Extract RDI values and calculate weights
def calculate_weights(df2_row):
    energy = df2_row['Energy (kcal)']
    weights = df2_row[1:].div(energy).mul(100).to_dict()
    return {key: round(value, 4) for key, value in weights.items()}

male_weights = calculate_weights(df2.iloc[0])
female_weights = calculate_weights(df2.iloc[1])

# Set specific weights for Energy (kcal)
male_weights['Energy (kcal)'] = 1.2
female_weights['Energy (kcal)'] = 0.9

# Calculate 60% RDA and 5% RDA values
rda_60_percent_male = df2.iloc[0, 1:] * 0.6
rda_5_percent_male = df2.iloc[0, 1:] * 0.05
rda_60_percent_female = df2.iloc[1, 1:] * 0.6
rda_5_percent_female = df2.iloc[1, 1:] * 0.05

# Penalty factors for specific nutrients
penalty_factors_exceeding = {
    'Energy (kcal)': 4,
    'Carbohydrate (g)': 5,
    'Sugars, total\n(g)': 7,
    'Total Fat (g)': 6,
    'Fatty acids, total saturated (g)': 8,
    'Fatty acids, total monounsaturated (g)': 2,
    'Fatty acids, total polyunsaturated (g)': 2,
    'Cholesterol (g)': 7,
    'Sodium (g)': 3,
}

# Penalty for deficiencies
penalty_factors_deficiency = {
    'Protein (g)': 10,
    'Fiber, total dietary (g)': 8,
    'Vitamin A, RAE (g)': 2,
    'Vitamin C (g)': 2,
    'Calcium (g)': 2,
    'Iron\n(g)': 2,
    'Carbohydrate (g)': 1,
    'Total Fat (g)': 2,
}

# Function to calculate weighted penalty
def calculate_weighted_penalty(row, rda_60_percent, rda_5_percent):
    # Calculate excess (for exceeding penalties) and deficiency (for deficiency penalties)
    excess = row[list(penalty_factors_exceeding.keys())] - rda_60_percent[list(penalty_factors_exceeding.keys())]
    deficiency = rda_5_percent[list(penalty_factors_deficiency.keys())] - row[list(penalty_factors_deficiency.keys())]
    
    # Apply penalties where criteria are met
    penalty_exceeding = (excess.clip(lower=0) * list(penalty_factors_exceeding.values())).sum()
    penalty_deficiency = (deficiency.clip(lower=0) * list(penalty_factors_deficiency.values())).sum()
    
    return 1 + penalty_exceeding + penalty_deficiency - 1  # Avoid double counting the base 1

# Function to calculate health scores
def calculate_health_scores(df, weights, rda_60_percent, rda_5_percent, gender):
    combined_weights = {key: default_weights[key] * weights.get(key, 1) for key in default_weights}

    # Calculate penalties
    penalty_factors_df = df[list(penalty_factors_exceeding.keys())] - rda_60_percent[list(penalty_factors_exceeding.keys())]
    penalty_factors_df = penalty_factors_df.clip(lower=0)
    
    deficiency_penalty_factors_df = rda_5_percent[list(penalty_factors_deficiency.keys())] - df[list(penalty_factors_deficiency.keys())]
    deficiency_penalty_factors_df = deficiency_penalty_factors_df.clip(lower=0)
    
    penalty = 1 + (penalty_factors_df * list(penalty_factors_exceeding.values())).sum(axis=1)
    deficiency_penalty = 1 + (deficiency_penalty_factors_df * list(penalty_factors_deficiency.values())).sum(axis=1)
    
    total_penalty = penalty + deficiency_penalty - 1  # Avoid double counting the base 1
    
    # Calculate health scores
    health_scores = (df[list(combined_weights.keys())] * pd.Series(combined_weights)).sum(axis=1) / total_penalty

    # Calculate percentile ranks
    health_scores_percentile = health_scores.rank(pct=True) * 100
    health_scores_percentile = health_scores_percentile.round(3)
    health_scores_percentile = health_scores_percentile.rank(method='first') * (100 / len(health_scores_percentile))

    return pd.DataFrame({
        'Food code': df['Food code'],
        'Main food description': df['Main food description'],
        f'Health Score {gender} Percentile': health_scores_percentile
    })

# Calculate health scores for males and females
df_male_scores = calculate_health_scores(df, male_weights, rda_60_percent_male, rda_5_percent_male, 'Male')
df_female_scores = calculate_health_scores(df, female_weights, rda_60_percent_female, rda_5_percent_female, 'Female')

# Merge the results
df = df.merge(df_male_scores, on=['Food code', 'Main food description'], how='left')
df = df.merge(df_female_scores, on=['Food code', 'Main food description'], how='left')

# Save the modified DataFrame back to a CSV file if needed
df.to_csv("Data_w_health_scores.csv", index=False)

# Example printout of the DataFrame with added health scores
print("DataFrame with Health Scores:")
print(df[['Food code', 'Main food description', 'Health Score Male Percentile', 'Health Score Female Percentile']].head(360))


DataFrame with Health Scores:
     Food code   Main food description  Health Score Male Percentile  \
0     11000000             Milk, human                     11.041963   
1     11100000               Milk, NFS                     18.527738   
2     11111000             Milk, whole                     20.288051   
3     11112110  Milk, reduced fat (2%)                     18.296586   
4     11112210      Milk, low fat (1%)                     17.140825   
..         ...                     ...                           ...   
355   14104200           Cheese, Colby                     23.737553   
356   14104250      Cheese, Colby Jack                     24.377667   
357   14104400            Cheese, Feta                     41.945235   
358   14104600         Cheese, Fontina                     26.760313   
359   14104700            Cheese, goat                     19.683499   

     Health Score Female Percentile  
0                         11.504267  
1                         18.